# Setup

## Set API key

In [1]:
import os   # type: ignore
import openai   # type: ignore

with open("../openai-api-key") as f:
    api_key = f.read()

openai.api_key = api_key   # type: ignore

#response = openai.Completion.create(engine="davinci", prompt="This is a test", max_tokens=5)

## Format Data

In [2]:
import pandas as pd
df = pd.read_csv('../Examples/RTA/example_outputs.csv', index_col=False)
df['rta'] = df['RTA_conf'].round().astype(bool)

print(df.columns)
print(df['rta'])

Index(['F1', 'AdmitDate', 'Day_In_Stay', 'RTA_conf', 'Date_Of_Process',
       'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5',
       'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9', 'Feature_10',
       'BWLoadDateTime', 'BWUpdateDateTime', 'Unnamed: 17', 'rta'],
      dtype='object')
0     True
1     True
2    False
3    False
Name: rta, dtype: bool


## Functions

### Creating a prompt

*Acute care*
> Acute care is a branch of secondary health care where a patient receives active but short-term treatment for a severe injury or episode of illness, an urgent medical condition, or during recovery from surgery. In medical terms, care for acute health conditions is the opposite from chronic care, or longer term care.

In [3]:
from typing import Optional

#TODO: Do not have to use "return to acute", "may have complications" is more general and the model might act better
# ^^^ from Elmaghraby 

def create_prompts(df:pd.DataFrame, seed_pre:Optional[str] = None, seed_post:Optional[str] = None):
    prompts = []
    for _, row in df.iterrows():
        prompt = ""

        if seed_pre is not None:
            prompt += seed_pre

        prompt += "It has been determined that the patient "

        if row['rta']:
            prompt += "may return to acute care " 
        else:
            prompt += "will not return to acute care "
        
        prompt += "with a confidence of {}. ".format(row['RTA_conf'])

        prompt += "This is due to the following factors: {}, {}, {}, {}, {}".format(
            row['Feature_1'],
            row['Feature_2'],
            row['Feature_3'],
            row['Feature_4'],
            row['Feature_5'],
        )
        
        if seed_post is not None:
            prompt += seed_post

        prompts.append(prompt)

    return prompts

def instruction_based_prompt(df, num_feats=5):
    assert num_feats > 0 and num_feats <= 10, "Must use between 1 and 10 features"
    prompts = []
    for _, row in df.iterrows():
        feats = list(row.iloc[5:5+num_feats])
        feats = [f.strip() for f in feats]
        prompt = "Explain to a nurse why a patient "
        if row['rta']:
            prompt += "has to return to acute care "
        else:
            prompt += "will not return to acute care "
        prompt += "based on the following features:\n\n"
        prompt += "Features:\n" + "\n".join(feats)
        prompt += "\n\nExplanation:\n\n"
        prompts.append(prompt)
    return prompts


def print_query(prompt, response):
    print("[PROMPT]:\n{}\n[RESPONSE]:\n{}".format(prompt, response['choices'][0]['text']))



# Query Experiments 

## Prompt Types

### TL;DR Summarization

In [4]:
prompts = create_prompts(df, seed_post="\n\ntl;dr:")
print(prompts[0])
# May, not "must"

It has been determined that the patient may return to acute care with a confidence of 0.89. This is due to the following factors:  CBC,  W/ AUTO DIFF,  PT EVALUATION MOD COMPLEXITY,  CULTURE SCREEN MRSA,  ARTERIAL BLOOD GAS

tl;dr:


### Summarize for a 2nd grader

In [79]:
prompts = create_prompts(df,
    seed_pre="My second grader asked me what this passage means:\n\"\"\"\n",
    seed_post="\n\"\"\"\nI rephrased it for him, in plain language a second grader can understand:\n\"\"\"\n"
)

print(prompts[0])

My second grader asked me what this passage means:
"""
It has been determined that the patient must return to acute care. This is due to the following factors:  CBC,  W/ AUTO DIFF,  PT EVALUATION MOD COMPLEXITY,  CULTURE SCREEN MRSA,  ARTERIAL BLOOD GAS
"""
I rephrased it for him, in plain language a second grader can understand:
"""



### Custom Instruction-based prompt

In [8]:
prompts = instruction_based_prompt(df, num_feats=10)
print(prompts[0])

Explain to a nurse why a patient has to return to acute care based on the following features:

Features:
CBC
W/ AUTO DIFF
PT EVALUATION MOD COMPLEXITY
CULTURE SCREEN MRSA
ARTERIAL BLOOD GAS
THERAPEUTIC PROCEDURE
EVAL SWALLOW/ORAL FUNCTION
VANCOMYCIN
GAIT TRAINING 15 MIN
METOPROLOL TART 25MG TAB

Explanation:




## Get Query Response

In [5]:
#engine="davinci"
# engine="ada"
#engine="curie-instruct-beta"
engine="davinci-instruct-beta"

num_token=100
#num_token=50

#temp = None
temp = 0.3

response = openai.Completion.create(engine=engine, prompt=prompts[0], max_tokens=num_token, temperature=temp)

In [6]:
print_query(prompts[0], response)
#print(response['choices'][0]['text'])



[PROMPT]:
It has been determined that the patient may return to acute care with a confidence of 0.89. This is due to the following factors:  CBC,  W/ AUTO DIFF,  PT EVALUATION MOD COMPLEXITY,  CULTURE SCREEN MRSA,  ARTERIAL BLOOD GAS

tl;dr:
[RESPONSE]:


The patient may return to acute care with a confidence of 0.89.

This is due to the following factors: CBC, W/ AUTO DIFF, PT EVALUATION MOD COMPLEXITY, CULTURE SCREEN MRSA, ARTERIAL BLOOD GAS


## Example outputs

### 2nd Grader Summarization with the curie-instruct-beta model

In [82]:
print_query(prompts[0], response)

Prompt:
My second grader asked me what this passage means:
"""
It has been determined that the patient must return to acute care. This is due to the following factors:  CBC,  W/ AUTO DIFF,  PT EVALUATION MOD COMPLEXITY,  CULTURE SCREEN MRSA,  ARTERIAL BLOOD GAS
"""
I rephrased it for him, in plain language a second grader can understand:
"""


Response:
The patient needs to go back to the hospital because they have a fever, a white blood cell count, and a urine test that showed they have an infection. 

I also explained that the word "due" means "because of" or


### TL;DR: with davinci-instruct-beta

In [94]:
print_query(prompts[0], response)

[PROMPT]:
It has been determined that the patient must return to acute care. This is due to the following factors:  CBC,  W/ AUTO DIFF,  PT EVALUATION MOD COMPLEXITY,  CULTURE SCREEN MRSA,  ARTERIAL BLOOD GAS

tl;dr:
[RESPONSE]:


The patient needs to go back to the hospital because they have a lot of problems.


### Instruction Prompt w/ Davinci-Instruct-Beta Model

In [123]:
print_query(prompts[0], response)

[PROMPT]:
Explain to a nurse why a patient may have to return to acute based on the following features:

Features:
CBC
W/ AUTO DIFF
PT EVALUATION MOD COMPLEXITY
CULTURE SCREEN MRSA
ARTERIAL BLOOD GAS
THERAPEUTIC PROCEDURE
EVAL SWALLOW/ORAL FUNCTION
VANCOMYCIN
GAIT TRAINING 15 MIN
METOPROLOL TART 25MG TAB

Explanation:


[RESPONSE]:


CBC: The patient's CBC may show an abnormality.

W/ AUTO DIFF: The patient may have an auto-immune disorder.

PT EVALUATION MOD COMPLEXITY: The patient may need to be evaluated for a more complex physical therapy evaluation.

CULTURE SCREEN MRSA: The patient may have a MRSA infection.

ARTERIAL BLOOD GAS: The patient may have an arterial blood gas that
